In [14]:

####################
#Author: brandon chiazza
#version: 1.0
#purpose: to call a twitter api and return results
#documentation: https://developer.twitter.com/en/docs
#####################

import pandas as pd
import requests
import json
import base64
import s3fs # documentation: https://s3fs.readthedocs.io/en/latest/
import time
import twitter_keys #this is a custom reference module to a package containing twitter keys

%config IPCompleter.greedy=True


key_secret = '{}:{}'.format(twitter_keys.client_key, twitter_keys.client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

#identify base url and oauth token path
base_url = 'https://api.twitter.com/' #base url for authentication
auth_url = '{}oauth2/token'.format(base_url)

#share header information -- encoding is ascii
auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

#pass clientcredentials
auth_data = {
    'grant_type': 'client_credentials'
}

#send authentication using requests - POST request
auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

#check response status. 200 = OK
auth_resp.status_code




200

In [15]:

#Keys in data response are token_type (bearer) and access_token (your access token)
print(auth_resp.json().keys())

access_token = auth_resp.json()['access_token']


search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}

#enter search parameters for coronavirus example. This looks for "covid-19" in the 1000 most recent tweets
query_params = {
    'q': '$SPAQ',
    'result_type': 'recent',
    'count': 100, #update here to get more/less than 1000 returns
    'lang': 'en' #filters by english language only
}


#identify search url path and save 
search_url = '{}1.1/search/tweets.json'.format(base_url)


#run search using get request
search_resp = requests.get(search_url, headers=search_headers, params=query_params)

#check status code of GET request
search_resp.status_code


dict_keys(['token_type', 'access_token'])


200

In [16]:
#print text from result to verify  
twitter_data = search_resp.json()

for x in twitter_data['statuses']:
    print(x['text'] + '\n')
    break #prints after one iteration and stops, remove break to see all 1000

@StockLizardKing $spaq it to me



In [17]:
# move data into data frame 
df = pd.DataFrame(twitter_data['statuses'])

# show one record to verify import 
df.head(10)

,created_at,id,id_str,text,truncated,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,...,favorite_count,favorited,retweeted,lang,extended_entities,retweeted_status,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,Sat Aug 08 01:06:03 +0000 2020,1291903466349236224,1291903466349236224,@StockLizardKing $spaq it to me,False,"{'hashtags': [], 'symbols': [{'text': 'spaq', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.291760e+18,1291760444789661696,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
1,Sat Aug 08 01:03:32 +0000 2020,1291902833667735552,1291902833667735552,This means we still have the OEM catalyst in a...,False,"{'hashtags': [], 'symbols': [{'text': 'spaq', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.291900e+18,1291900276664930304,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
2,Sat Aug 08 01:02:07 +0000 2020,1291902477126819845,1291902477126819845,RT @AlanWeedspan: holding through $SPAQ shake ...,False,"{'hashtags': [], 'symbols': [{'text': 'SPAQ', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,0,False,False,en,"{'media': [{'id': 1291446763073810434, 'id_str...",{'created_at': 'Thu Aug 06 18:51:18 +0000 2020...,False,NaN,NaN,NaN
3,Sat Aug 08 01:02:02 +0000 2020,1291902457845620736,1291902457845620736,@QStonks Magna has their own electric drivetra...,False,"{'hashtags': [], 'symbols': [{'text': 'SPAQ', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.291900e+18,1291900276664930304,...,1,False,False,en,NaN,NaN,False,NaN,NaN,NaN
4,Sat Aug 08 01:00:46 +0000 2020,1291902137367224320,1291902137367224320,RT @StockLizardKing: $SPAQ just on Fast Money ...,False,"{'hashtags': [], 'symbols': [{'text': 'SPAQ', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,0,False,False,en,"{'media': [{'id': 1291850487655010305, 'id_str...",{'created_at': 'Fri Aug 07 21:36:05 +0000 2020...,False,NaN,NaN,NaN
5,Sat Aug 08 01:00:01 +0000 2020,1291901946719240192,1291901946719240192,$idex $fb $spy $tsla $nkla $aapl $aal $vxrt $n...,True,"{'hashtags': [], 'symbols': [{'text': 'idex', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
6,Sat Aug 08 00:59:50 +0000 2020,1291901900250648577,1291901900250648577,Strong close to end the week $JMIA $NAK $EVFM ...,False,"{'hashtags': [], 'symbols': [{'text': 'JMIA', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,0,False,False,en,NaN,NaN,NaN,NaN,NaN,NaN
7,Sat Aug 08 00:59:27 +0000 2020,1291901807338430464,1291901807338430464,RT @sluttymario: Tip of the day for new trader...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,0,False,False,en,NaN,{'created_at': 'Sat Aug 08 00:23:32 +0000 2020...,NaN,NaN,NaN,NaN
8,Sat Aug 08 00:54:36 +0000 2020,1291900585487118337,1291900585487118337,RT @aaaamhim: $RLFTF added again I love this a...,False,"{'hashtags': [], 'symbols': [{'text': 'RLFTF',...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,0,False,False,en,NaN,{'created_at': 'Fri Aug 07 19:53:25 +0000 2020...,NaN,NaN,NaN,NaN
9,Sat Aug 08 00:53:22 +0000 2020,1291900276664930304,1291900276664930304,Seeing a lot of mis-info re: $spaq and the mag...,True,"{'hashtags': [], 'symbols': [{'text': 'spaq', ...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,None,...,3,False,False,en,NaN,NaN,False,

In [18]:
# we can use pandas to put data directly into an s3 bucket
#prepare csv file name   
filename = 's3:/lab-03/'#specify location of s3:/{my-bucket}/
groupname= 'Haris_Sumra' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(filename,groupname,datetime) #name of the filepath and csv file



#encoding must be adjusted to accommodate abnormal characters. Use s3fs to write to S3 bucket
byte_encoded_df = df.to_csv(None).encode() #encodes file as binary
s3 = s3fs.S3FileSystem(anon=False)
with s3.open(filenames3, 'wb') as file:
    file.write(byte_encoded_df) #writes byte-encoded file to s3 location

#print success message
print("Successfull uploaded file to location:"+str(filenames3))


NoCredentialsError: Unable to locate credentials